## Import necessary packages

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4,5,6,7"
from kilt import retrieval
from kilt import kilt_utils as utils
import tasks
# from kilt.retrievers import DPR_connector
import utils
from rouge_score import rouge_scorer
import random
import numpy as np
import torch
torch.set_grad_enabled(False)
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from datasets import load_dataset
import json
from tqdm import tqdm
import opensource

np.random.seed(21)

In [2]:
import pickle
def write_list(a_list, file_name):
    # store list in binary file so 'wb' mode
    with open(file_name, 'wb') as fp:
        pickle.dump(a_list, fp)
        print('Done writing list into a binary file')
def read_list(file_name):
    # for reading also binary mode is important
    with open(file_name, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list

def save_results(task):
    # save retrieved_scores to a pickle file
    write_list(retrieved_scores, f'retrieved_scores_{task}.p')
    # save retrieved_true_scores to a pickle file
    write_list(retrieved_true_scores, f'retrieved_true_scores_{task}.p')
    # save queries to a pickle file
    write_list(queries, f'queries_{task}.p')
    # save answers to a pickle file
    write_list(answers, f'answers_{task}.p')
    # save passages to a pickle file
    write_list(passages, f'passages_{task}.p')
    # save opensource_true_scores to a pickle file
    write_list(opensource_true_scores, f'opensource_true_scores_{task}.p')
    # save opensource_texts to a ickle file
#     write_list(opensource_texts, f'opensource_texts_{task}.p')
    # save opensource_answers to a pickle file
    write_list(opensource_answers, f'opensource_answers_{task}.p')
    # save opensource_semantics to a picle file
    write_list(opensource_semantics, f'opensource_semantics_{task}.p')
    # save feasibilities to a pickle file
    write_list(feasibilities, f'feasibilities_{task}.p')
    # save occurances to a pickle file
    write_list(occurances, f'occurances_{task}.p')
    # save semantic_ids to a pickle file
    write_list(semantic_ids, f'semantic_ids_{task}.p')

def read_results(task, end=None):
    retrieved_scores = read_list(f'retrieved_scores_{task}.p')[:end]
    retrieved_true_scores = read_list(f'retrieved_true_scores_{task}.p')[:end]
    queries = read_list(f'queries_{task}.p')[:end]
    answers = read_list(f'answers_{task}.p')[:end]
    opensource_true_scores = read_list(f'opensource_true_scores_{task}.p')[:end]
    opensource_answers = read_list(f'opensource_answers_{task}.p')[:end]
    opensource_semantics = read_list(f'opensource_semantics_{task}.p')[:end]
    opensource_occurances = read_list(f'occurances_{task}.p')[:end]
    opensource_semantic_ids = read_list(f'semantic_ids_{task}.p')[:end]
    
    return retrieved_scores, retrieved_true_scores, \
           queries, answers, \
           opensource_true_scores, opensource_answers, \
           opensource_occurances, opensource_semantic_ids

In [28]:
task = 'squad1'

In [19]:
semantic = True
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"],
                                        use_stemmer=True)
if semantic:
    from transformers import AutoModelForSequenceClassification, AutoTokenizer
    # setup semantic model
    semantic_tokenizer = \
        AutoTokenizer.from_pretrained("microsoft/deberta-large-mnli")
    semantic_model = \
        AutoModelForSequenceClassification.from_pretrained(
            "microsoft/deberta-large-mnli"
        ).cuda()

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
retrieved_scores, retrieved_true_scores, queries, answers, opensource_true_scores, opensource_answers, opensource_occurances, opensource_semantic_ids = \
        read_results(task, end=1004)

In [30]:
len(queries)

1004

In [31]:
answers_semantic = []
for true_score, scores, returned_answers in zip(retrieved_true_scores, retrieved_scores, opensource_answers):
    idx = list(scores).index(true_score)
    tmp = returned_answers[idx]
    answers_semantic.append(tmp)

In [32]:
def compute_semantic_clusterring(model, tokenizer,
                                 question, answers):
    # filter out non-unique answers
    unique_generated_texts, item_occurance = \
        utils.unique_items(answers)

    # unique_generated_texts = list(set(answers))
    semantic_set_ids = {}
    for index, answer in enumerate(unique_generated_texts):
        semantic_set_ids[answer] = index
    # print('Number of unique answers:', len(unique_generated_texts))

    with torch.no_grad():
        semantics = [0]
        for i in range(1, len(unique_generated_texts)):
            included = False
            for semantic in semantics:
                qa_1 = question + ' ' + unique_generated_texts[semantic]
                qa_2 = question + ' ' + unique_generated_texts[i]

                input = qa_1 + ' [SEP] ' + qa_2
                encoded_input = tokenizer.encode(input, padding=True)
                prediction = model(torch.tensor([encoded_input], device='cuda'))['logits']
                predicted_label = torch.argmax(prediction, dim=1)

                reverse_input = qa_2 + ' [SEP] ' + qa_1
                encoded_reverse_input = tokenizer.encode(
                    reverse_input,
                    padding=True)
                reverse_prediction = model(torch.tensor([encoded_reverse_input], device='cuda'))['logits']
                reverse_predicted_label = torch.argmax(
                    reverse_prediction,
                    dim=1)

                if 2 in predicted_label and 2 in reverse_predicted_label:
                    semantic_set_ids[unique_generated_texts[i]] = \
                        semantic_set_ids[unique_generated_texts[semantic]]
                    included = True
                    break
            if not included:
                semantics.append(i)

    semantic_probs = utils.get_semantic_prob(semantic_set_ids, item_occurance)
    return semantic_set_ids, semantic_probs, item_occurance

In [33]:
opensource_true_scores_semantic = []
skip = []
for idx, [query_tmp, true_answer, generated_answer] in tqdm(enumerate(zip(queries, answers, answers_semantic)), total=len(queries)):

    prompt = utils.get_prompt_template(query_tmp, "", task="Natural Questions")
    semantic_set_ids, semantic_probs, item_occurance = \
        compute_semantic_clusterring(
            model=semantic_model,
            tokenizer=semantic_tokenizer,
            question=query_tmp,
            answers=generated_answer
        )
    true_scores, matched_answer, semantics = utils.processing_answers(
        semantic_set_ids, semantic_probs, 
        item_occurance, true_answer, scorer,
        threshold=0.3
    )
    if len(true_scores) == 0:
        print(idx)
        skip.append(idx)
        continue
    opensource_true_scores_semantic.append(true_scores)

 49%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                                                                                                                                                                             | 488/1004 [01:10<01:36,  5.33it/s]

488


 75%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                   | 754/1004 [02:23<00:46,  5.38it/s]

758


 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋      | 988/1004 [03:14<00:08,  1.81it/s]

987


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1004/1004 [03:15<00:00,  5.13it/s]


In [ ]:
occurances_semantic = []
semantic_ids_semantic = []
probs_semantic = []
for idx, [query_tmp, generated_answers] in tqdm(enumerate(zip(queries, opensource_answers)), total=len(queries)):
    if idx in skip:
        continue
    probs_tmp = []
    answers_tmp = []
    semantic_id_tmp = []
    occurance_tmp = []
    semantic_tmp = []
    for generated_answer in generated_answers:
        prompt = utils.get_prompt_template(query_tmp, "", task="Natural Questions")
        semantic_set_ids, semantic_probs, item_occurance = \
            compute_semantic_clusterring(
            answers=generated_answer, 
            question=query_tmp,
            model=semantic_model,
            tokenizer=semantic_tokenizer,
        )
        probs_tmp.append(semantic_probs)
        occurance_tmp.append(item_occurance)
        semantic_id_tmp.append(semantic_set_ids)
    occurances_semantic.append(occurance_tmp)
    semantic_ids_semantic.append(semantic_id_tmp)
    probs_semantic.append(probs_tmp)

  9%|██████████████████████████████████▍                                                                                                                                                                                                                                                                                                                                                                           | 87/1004 [28:10<2:33:44, 10.06s/it]

In [38]:
write_list(opensource_true_scores_semantic, f'opensource_true_scores_{task}_semantic.p')
write_list(occurances_semantic, f'opensource_occurances_{task}_semantic.p')
write_list(semantic_ids_semantic, f'opensource_semantic_ids_{task}_semantic.p')
write_list(probs_semantic, f'opensource_probs_{task}_semantic.p')

Done writing list into a binary file
Done writing list into a binary file
Done writing list into a binary file
Done writing list into a binary file


In [39]:
opensource_true_scores_semantic = read_list(f'opensource_true_scores_{task}_semantic.p')
opensource_occurances_semantic = read_list(f'opensource_occurances_{task}_semantic.p')
opensource_semantic_ids_semantic = read_list(f'opensource_semantic_ids_{task}_semantic.p')
opensource_probs_semantic = read_list(f'opensource_probs_{task}_semantic.p')

In [ ]:
# indices = np.arange(1000)
indices = np.arange(len(queries))
random.shuffle(indices)
cal_first_indices = indices[:int(len(indices) * 0.3)]
cal_second_indices = indices[int(len(indices) * 0.3) : int(len(indices) * 0.6)]
test_indices = indices[int(len(indices) * 0.6):]
# test_indices = indices[int(len(indices) * 0.3):]

cal_first_queries = utils.split(queries, cal_first_indices)
cal_second_queries = utils.split(queries, cal_second_indices)
test_queries = utils.split(queries, test_indices)

cal_first_answers = utils.split(answers, cal_first_indices)
cal_second_answers = utils.split(answers, cal_second_indices)
test_answers = utils.split(answers, test_indices)

cal_first_answers_semantic = utils.split(answers_semantic, cal_first_indices)
cal_second_answers_semantic = utils.split(answers_semantic, cal_second_indices)
test_answers_semantic = utils.split(answers_semantic, test_indices)

cal_first_retrieved_true_scores = utils.split(retrieved_true_scores, cal_first_indices)
cal_second_retrieved_true_scores = utils.split(retrieved_true_scores, cal_second_indices)
test_retrieved_true_scores = utils.split(retrieved_true_scores, test_indices)

cal_first_opensource_true_scores = utils.split(opensource_true_scores, cal_first_indices)
cal_second_opensource_true_scores = utils.split(opensource_true_scores, cal_second_indices)
test_opensource_true_scores = utils.split(opensource_true_scores, test_indices)

cal_first_retrieved_scores = utils.split(retrieved_scores, cal_first_indices)
cal_second_retrieved_scores = utils.split(retrieved_scores, cal_second_indices)
test_retrieved_scores = utils.split(retrieved_scores, test_indices)

cal_first_opensource_occurances = utils.split(opensource_occurances, cal_first_indices)
cal_second_opensource_occurances = utils.split(opensource_occurances, cal_second_indices)
test_opensource_occurances = utils.split(opensource_occurances, test_indices)

cal_first_opensource_semantic_ids = utils.split(opensource_semantic_ids, cal_first_indices)
cal_second_opensource_semantic_ids = utils.split(opensource_semantic_ids, cal_second_indices)
test_opensource_semantic_ids = utils.split(opensource_semantic_ids, test_indices)

cal_first_queries = utils.split(queries, cal_first_indices)
cal_second_queries = utils.split(queries, cal_second_indices)
test_queries = utils.split(queries, test_indices)

cal_first_opensource_answers = utils.split(opensource_answers, cal_first_indices)
cal_second_opensource_answers = utils.split(opensource_answers, cal_second_indices)
test_opensource_answers = utils.split(opensource_answers, test_indices)

cal_first_answers = utils.split(answers, cal_first_indices)
cal_second_answers = utils.split(answers, cal_second_indices)
test_answers = utils.split(answers, test_indices)

In [ ]:
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"],
                                        use_stemmer=True)

## Compute coverage rate

In [ ]:
def coverage(
        retrieved_true_scores_list, opensource_true_scores_list,
        retrieved_thr, qa_thr):

    includes = []
    for idx, (retrieved_true_score, opensource_true_score) in enumerate(zip(retrieved_true_scores_list, opensource_true_scores_list)):
#         if idx > 20:
        opensource_true_score = np.max(opensource_true_score)
        include = True if (retrieved_true_score >= retrieved_thr and 
                           opensource_true_score >= qa_thr) \
                       else False
        includes.append(include)
    return includes

In [ ]:
alpha = 0.1
retrieved_thr = utils.compute_threshold(cal_first_retrieved_true_scores, alpha=alpha/2)
cal_first_scores = []
for scores in cal_first_opensource_true_scores:
    cal_first_scores.append(np.max(scores))
opensource_qa_thr = utils.compute_threshold(cal_first_scores, alpha=alpha/2)

retrieved_coverage = np.mean(np.array(cal_second_retrieved_true_scores) >= retrieved_thr)
cal_second_scores = []
for scores in cal_second_opensource_true_scores:
    cal_second_scores.append(np.max(scores))
qa_coverage = np.mean(np.array(cal_second_scores) >= opensource_qa_thr)
print('retrieval coverage', retrieved_coverage)
print('qa coverage', qa_coverage)

retrieved_coverage = np.mean(np.array(test_retrieved_true_scores) >= retrieved_thr)
test_scores = []
for scores in test_opensource_true_scores:
    test_scores.append(np.max(scores))
qa_coverage = np.mean(np.array(test_scores) >= opensource_qa_thr)
print('test retrieval coverage', retrieved_coverage)
print('test qa coverage', qa_coverage)

In [ ]:
coverages = coverage(test_retrieved_true_scores, 
                    test_opensource_true_scores,
                    retrieved_thr,
                    opensource_qa_thr
                   )
print('End-to-end coverage', np.mean(coverages))

## PAC-Bonf

In [ ]:
from pac_utils import find_maximum_train_error_allow

In [ ]:
delta = 0.1
retrieve_alpha = find_maximum_train_error_allow(alpha/2.0, delta/2.0, len(cal_first_indices))
qa_alpha = find_maximum_train_error_allow(alpha/2.0, delta/2.0, len(cal_first_indices))
retrieved_thr = utils.compute_threshold(cal_first_retrieved_true_scores, alpha=retrieve_alpha)
cal_first_scores = []
for scores in cal_first_opensource_true_scores:
    cal_first_scores.append(np.max(scores))
opensource_qa_thr = utils.compute_threshold(cal_first_scores, alpha=qa_alpha)

retrieved_coverage = np.mean(np.array(cal_second_retrieved_true_scores) >= retrieved_thr)
cal_second_scores = []
for scores in cal_second_opensource_true_scores:
    cal_second_scores.append(np.max(scores))
qa_coverage = np.mean(np.array(cal_second_scores) >= opensource_qa_thr)
print('retrieval coverage', retrieved_coverage)
print('qa coverage', qa_coverage)

retrieved_coverage = np.mean(np.array(test_retrieved_true_scores) >= retrieved_thr)
test_scores = []
for scores in test_opensource_true_scores:
    test_scores.append(np.max(scores))
qa_coverage = np.mean(np.array(test_scores) >= opensource_qa_thr)
print('test retrieval coverage', retrieved_coverage)
print('test qa coverage', qa_coverage)

In [ ]:
coverages = coverage(test_retrieved_true_scores, 
                    test_opensource_true_scores,
                    retrieved_thr,
                    opensource_qa_thr
                   )
print('End-to-end coverage', np.mean(coverages))

## Bayesian optimization

In [ ]:
from skopt.space import Real
from skopt import gp_minimize
from skopt.utils import use_named_args

In [ ]:
def evaluate(
        test_retrieved_scores,
        test_queries, test_answers, test_chatgpt_answers, 
        test_chatgpt_semantic_ids, test_chatgpt_probs,
        retrieved_thr, chatgpt_qa_thr,
        cluster=True, kernel=40, verbose=True):
    
    length = len(test_retrieved_scores)
    lens = np.linspace(0, length, kernel+1)
    test_retrieved_scores_list = [test_retrieved_scores[int(lens[i]):int(lens[i+1])] for i in range(kernel)]
    test_chatgpt_semantic_ids_list = [test_chatgpt_semantic_ids[int(lens[i]):int(lens[i+1])] for i in range(kernel)]
    test_chatgpt_probs_list = [test_chatgpt_probs[int(lens[i]):int(lens[i+1])] for i in range(kernel)]
    test_answers_list = [test_answers[int(lens[i]):int(lens[i+1])] for i in range(kernel)]
    
    def run(i, shared_includes, shared_semantic_counts):
        includes = []
        semantics_total = []
        coverages = []
        semantic_counts = []
        for idx, (retrieved_scores_tmp, answers_tmp,\
                chatgpt_semantic_ids_tmp, chatgpt_probs_tmp) \
            in tqdm(enumerate(zip(
                test_retrieved_scores_list[i], test_answers_list[i], \
                test_chatgpt_semantic_ids_list[i], test_chatgpt_probs_list[i])), total=len(test_retrieved_scores_list[i])):

            include = False
    #         coverage_tmp = coverage(retrieved_true_scores_tmp, 
    #                         [chatgpt_true_scores_tmp],
    #                         retrieved_thr,
    #                         chatgpt_qa_thr
    #                        )[0]
    #         coverages.append(coverage_tmp)
            retrieved_count = 0
            semantics = []
            for retrieved_score, semantic_set_ids, probs in zip(retrieved_scores_tmp, chatgpt_semantic_ids_tmp, chatgpt_probs_tmp):
                    if retrieved_score < retrieved_thr:
                        continue
                    else:
                        retrieved_count += 1
                        for predicted_answer in semantic_set_ids.keys():
                            concept_id = semantic_set_ids[predicted_answer]
                            prob = probs[concept_id]
                            if prob >= chatgpt_qa_thr:
                                semantics.append(predicted_answer)

                                # TODO: check if the semantic is consistent with true answer
                                if include is False:
                                    for answer_tmp in answers_tmp:
                                        scores = scorer.score(answer_tmp,
                                                              predicted_answer)
                                        scores = scores['rouge1'][2]
                                        if scores >= 0.3:
                                            include = True
                                            break
            if cluster:
                semantic_set_ids, semantic_probs, item_occurance = \
                            utils.clustering(semantics, "", scorer=scorer)
                semantic_counts.append(len(semantic_probs.keys()))
            else:
                semantic_counts.append(len(semantics))
            semantics_total.append(semantic_set_ids)
    #         answer_counts.append(retrieved_count)
            includes.append(include)
        shared_includes.value += np.sum(includes)
        shared_semantic_counts.value += np.sum(semantic_counts)
    
    processes = []
    shared_includes = Value('f', 0.0)
    shared_semantic_counts = Value('f', 0.0)

    for i in range(0, kernel):
        p = multiprocessing.Process(target=run, args=(i, shared_includes, shared_semantic_counts))
        processes.append(p)
        p.start()

    for process in processes:
        process.join()
        
    return shared_includes.value/length, 0.0, shared_semantic_counts.value/length

## Bonf method

In [ ]:
def softmax(vec):
    nom = np.exp(vec - np.mean(vec))
    return nom / np.sum(nom)

In [ ]:
"""
Weight HMP module
"""
w1 = Real(name='w1', low=0.0, high=1.0)
w2 = Real(name='w2', low=0.0, high=1.0)

# Gather the search-space dimensions in a list.
dimensions = [w1, w2]

@use_named_args(dimensions=dimensions)
def objective(w1, w2):
    weights = softmax(np.array([w1, w2])).reshape(-1, 1)
    alpha_retrieve = alpha * weights[0]
    alpha_qa = alpha * weights[1]
    retrieved_thr = utils.compute_threshold(cal_first_retrieved_true_scores, alpha=alpha_retrieve)
    cal_first_scores = []
    for scores in cal_first_opensource_true_scores:
        cal_first_scores.append(np.max(scores))
    opensource_qa_thr = utils.compute_threshold(cal_first_scores, alpha=alpha_qa)
    results = evaluate(cal_second_retrieved_scores, cal_second_queries, 
                       cal_second_answers, cal_second_opensource_answers, \
                       cal_second_opensource_occurances, cal_second_opensource_semantic_ids,
                       retrieved_thr, opensource_qa_thr,
                       cluster=True
                      )
    coverage = np.mean(results[0])
    average_answer = np.mean(results[1])
    average_semantic = np.mean(results[2])
    return average_semantic

result = gp_minimize(func=objective,
                     dimensions=dimensions,
                     acq_func="EI",      # the acquisition function
                     n_calls=10,
                     random_state=42,
                     verbose=True)

print("Best fitness:", result.fun)
print("Best parameters:", softmax(result.x))

In [ ]:
weights = softmax(result.x).reshape(-1, 1)
alpha_retrieve = alpha * weights[0]
alpha_qa = alpha * weights[1]
retrieved_thr = utils.compute_threshold(cal_first_retrieved_true_scores, alpha=alpha_retrieve)
cal_first_scores = []
for scores in cal_first_opensource_true_scores:
    cal_first_scores.append(np.max(scores))
opensource_qa_thr = utils.compute_threshold(cal_first_scores, alpha=alpha_qa)
results = evaluate(
    test_retrieved_scores, test_queries,
    test_answers, test_opensource_answers, 
    test_opensource_occurances, test_opensource_semantic_ids,
    retrieved_thr, opensource_qa_thr,
    cluster=True
)

In [ ]:
print('Desired Coverage', 1-alpha)
print('Coverage', np.mean(results[0]))
# print('Average answer', np.mean(results[1]))
print('Average semantic', np.mean(results[2]))

In [ ]:
alpha_retrieve = alpha * (1/2.0)
alpha_qa = alpha * (1/2.0)
retrieved_thr = utils.compute_threshold(cal_first_retrieved_true_scores, alpha=alpha_retrieve)
cal_first_scores = []
for scores in cal_first_opensource_true_scores:
    cal_first_scores.append(np.max(scores))
opensource_qa_thr = utils.compute_threshold(cal_first_scores, alpha=alpha_qa)
results = evaluate(
    test_retrieved_scores, test_queries,
    test_answers, test_opensource_answers, 
    test_opensource_occurances, test_opensource_semantic_ids,
    retrieved_thr, opensource_qa_thr,
    cluster=True
)

In [ ]:
print('Desired Coverage', 1-alpha)
print('Coverage', np.mean(results[0]))
# print('Average answer', np.mean(results[1]))
print('Average semantic', np.mean(results[2]))

In [ ]:
def evaluate_vanila(
        test_retrieved_scores,
        test_queries, test_answers, test_chatgpt_answers, 
        test_chatgpt_semantic_ids, test_chatgpt_probs,
        cluster=True, kernel=40, verbose=True):
    
    length = len(test_retrieved_scores)
    lens = np.linspace(0, length, kernel+1)
    test_retrieved_scores_list = [test_retrieved_scores[int(lens[i]):int(lens[i+1])] for i in range(kernel)]
    test_chatgpt_semantic_ids_list = [test_chatgpt_semantic_ids[int(lens[i]):int(lens[i+1])] for i in range(kernel)]
    test_chatgpt_probs_list = [test_chatgpt_probs[int(lens[i]):int(lens[i+1])] for i in range(kernel)]
    test_answers_list = [test_answers[int(lens[i]):int(lens[i+1])] for i in range(kernel)]
    
    def run(i, shared_includes, shared_semantic_counts):
        includes = []
        semantics_total = []
        coverages = []
        semantic_counts = []
        for idx, (retrieved_scores_tmp, answers_tmp,\
                chatgpt_semantic_ids_tmp, chatgpt_probs_tmp) \
            in tqdm(enumerate(zip(
                test_retrieved_scores_list[i], test_answers_list[i], \
                test_chatgpt_semantic_ids_list[i], test_chatgpt_probs_list[i])), total=len(test_retrieved_scores_list[i])):

            include = False
    #         coverage_tmp = coverage(retrieved_true_scores_tmp, 
    #                         [chatgpt_true_scores_tmp],
    #                         retrieved_thr,
    #                         chatgpt_qa_thr
    #                        )[0]
    #         coverages.append(coverage_tmp)
            retrieved_count = 0
            semantics = []
            for retrieved_score, semantic_set_ids, probs in zip(retrieved_scores_tmp, chatgpt_semantic_ids_tmp, chatgpt_probs_tmp):
                retrieved_count += 1
                for predicted_answer in semantic_set_ids.keys():
                    concept_id = semantic_set_ids[predicted_answer]
                    prob = probs[concept_id]
                    semantics.append(predicted_answer)

                    # TODO: check if the semantic is consistent with true answer
                    if include is False:
                        for answer_tmp in answers_tmp:
                            scores = scorer.score(answer_tmp,
                                                  predicted_answer)
                            scores = scores['rouge1'][2]
                            if scores >= 0.3:
                                include = True
                                break
                break
            if cluster:
                semantic_set_ids, semantic_probs, item_occurance = \
                            utils.clustering(semantics, "", scorer=scorer)
                semantic_counts.append(len(semantic_probs.keys()))
            else:
                semantic_counts.append(len(semantics))
            semantics_total.append(semantic_set_ids)
    #         answer_counts.append(retrieved_count)
            includes.append(include)
        shared_includes.value += np.sum(includes)
        shared_semantic_counts.value += np.sum(semantic_counts)
    
    processes = []
    shared_includes = Value('f', 0.0)
    shared_semantic_counts = Value('f', 0.0)

    for i in range(0, kernel):
        p = multiprocessing.Process(target=run, args=(i, shared_includes, shared_semantic_counts))
        processes.append(p)
        p.start()

    for process in processes:
        process.join()
        
    return shared_includes.value/length, 0.0, shared_semantic_counts.value/length

In [ ]:
%%time
results = evaluate_vanila(
    test_retrieved_scores, test_queries,
    test_answers, test_opensource_answers, 
    test_opensource_occurances, test_opensource_semantic_ids,
    cluster=True)

In [ ]:
print('Desired coverage rate', 1-alpha)
print('Coverage', np.mean(results[0]))
print('Average answer', np.mean(results[1]))
print('Average semantic', np.mean(results[2]))

## PAC-Bonf method

In [ ]:
alpha_retrieve = alpha * (1/2.0)
alpha_qa = alpha * (1/2.0)

delta = 0.1
alpha_retrieve_pac = find_maximum_train_error_allow(alpha_retrieve, delta/2.0, len(cal_first_indices))
alpha_qa_pac = find_maximum_train_error_allow(alpha_qa, delta/2.0, len(cal_first_indices))

retrieved_thr = utils.compute_threshold(cal_first_retrieved_true_scores, alpha=alpha_retrieve_pac)
cal_first_scores = []
for scores in cal_first_opensource_true_scores:
    cal_first_scores.append(np.max(scores))
opensource_qa_thr = utils.compute_threshold(cal_first_scores, alpha=alpha_qa_pac)
results = evaluate(
    test_retrieved_scores, test_queries,
    test_answers, test_opensource_answers, 
    test_opensource_occurances, test_opensource_semantic_ids,
    retrieved_thr, opensource_qa_thr)

In [ ]:
print('Desired Coverage', 1-alpha)
print('Coverage', np.mean(results[0]))
# print('Average answer', np.mean(results[1]))
print('Average semantic', np.mean(results[2]))